# NYPD Civilian Complaints

# Data Sources

Source for current number of Uniformed officers: https://council.nyc.gov/budget/wp-content/uploads/sites/54/2019/03/056-NYPD-2020.pdf<br>
Civilian Complaints Dataset: https://www.propublica.org/datastore/dataset/civilian-complaints-against-new-york-city-police-officers<br>
Police Reports Dataset: https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Current-Year-To-Date-/5uac-w243<br>
Precinct Geographic Data: https://data.cityofnewyork.us/Public-Safety/Police-Precincts/78dh-3ptz<br>

# Initialize

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
import folium
import os
import folium.plugins as plugins
import geopandas as gpd

alt.renderers.enable('notebook')
alt.data_transformers.disable_max_rows()

#Setup altair to save graphs to external file. This breaks Vega but is needed if notebook gets too large.

#def json_dir(data, data_dir = 'altairdata'):
    #os.makedirs(data_dir, exist_ok = True)
    #return alt.pipe(data, alt.to_json(filename = data_dir + '/{prefix}-{hash}.{extension}'))
#alt.data_transformers.register('json_dir', json_dir)
#alt.data_transformers.enable('json_dir', data_dir = 'mydata')

DataTransformerRegistry.enable('default')

# Pull in data

In [13]:
df = pd.read_csv('Complaints.csv')
policeReports = pd.read_csv('NYPD_Complaint_Data_Current__Year_To_Date_.csv', low_memory = False)
precinctMap = gpd.read_file('PolicePrecincts.geojson')

# Clean Data

In [14]:
df['complainant_ethnicity'] = df['complainant_ethnicity'].fillna('Unknown')
df['command_at_incident'] = df['command_at_incident'].fillna('Unknown')
df = df.replace({'precinct': {1000:22}})

# Format Data

In [15]:
# Join arrest report data with precinct outline data for folium
geoPoliceReports = gpd.GeoDataFrame(policeReports, geometry = gpd.points_from_xy(policeReports.Longitude,policeReports.Latitude))
geoPoliceReports.crs = precinctMap.crs
geoPoliceReports = gpd.sjoin(geoPoliceReports,precinctMap,how = 'inner')

In [16]:
# Start dataframe indexed by precincts to get data for each precinct
precincts = pd.DataFrame(columns = ['precinct'])
precincts['precinct'] = np.sort(df['precinct'].unique())
precincts = precincts.join(df.groupby(['precinct']).size().to_frame(name = 'Number of Complaints'),on = 'precinct')

# Setting up dict to attach number of uniformed officers to precinct
keys = precincts['precinct'].unique()

# Values for current number of Uniformed officers taken from NYPD preliminary budget report
values = [190,141,164,142,178,162,190,307,141,294,207,
         149,120,176,156,210,134,167,142,205,191,201,
         284,186,221,293,358,174,306,280,252,193,133,
         282,202,167,154,155,138,333,138,161,303,216,
         179,275,399,139,235,198,247,197,209,221,155,
         185,142,145,206,209,254,182,310,225,197,148,
         292,211,153,144,236,261,234,311,196,195,144]

In [17]:
# Function to attach number of uniformed officers to precinct
UniformedDict = dict(zip(keys,values))
def amtUniformed(precinct):
    return UniformedDict[precinct]

In [18]:
# Build out precinct dataframe
precincts['Uniformed Officers'] = precincts['precinct']
precincts['Uniformed Officers'] = precincts['Uniformed Officers'].apply(amtUniformed)
precincts['Complaints per Officer'] = precincts['Number of Complaints'] / precincts['Uniformed Officers']

In [19]:
# Get race statistics of different precincts using police report dataset
minoritydf = pd.pivot_table(geoPoliceReports,
                            values = ['Latitude'], 
                            index = ['precinct'], 
                            columns = ['VIC_RACE'], 
                            aggfunc = 'count')
minoritydf['total'] = minoritydf.sum(1)
minoritydf['minority total'] = minoritydf['Latitude'][['BLACK','BLACK HISPANIC','WHITE HISPANIC']].sum(1)
minoritydf = minoritydf.drop(['Latitude'], axis = 1)
minoritydf['ratio'] = minoritydf['minority total']/minoritydf['total']
minoritydf = minoritydf.reset_index()

# Get names of board outcomes and make dataframe for outcomes per precinct
substantiatedCols = df['board_disposition'].unique()
substantiatedCols = substantiatedCols[(substantiatedCols != 'Exonerated') & (substantiatedCols != 'Unsubstantiated')]
outdf = pd.pivot_table(df,
                       values = ['first_name'],
                       index = ['precinct'],
                       columns = ['board_disposition'],
                       aggfunc = 'count')['first_name']
outdf = outdf.assign(Substantiated = outdf[substantiatedCols].sum(1)).drop(substantiatedCols,1)

# Make and clean dataframe for year range of complaints for each precinct
yeardf = pd.pivot_table(df,
                        values = ['year_received'],
                        index = ['precinct'],
                        aggfunc = ['min','max'])
yeardf['min year'] = yeardf['min']['year_received']
yeardf['max year'] = yeardf['max']['year_received']
yeardf = yeardf.drop('min',1)
yeardf = yeardf.drop('max',1)
yeardf.columns = yeardf.columns.droplevel(1)

# Dataframe of stats based on each officer
uniqueIDdf = pd.pivot_table(df,
                            values = ['first_name','year_received'], 
                            index = ['precinct','unique_mos_id'],
                            aggfunc ={'first_name':'count','year_received':'max'}).sort_values(by = 'first_name')

# Join all dataframes to precinct dataframe
precincts = precincts.join(outdf, on = 'precinct', how = 'inner')
precincts = precincts.join(yeardf, on = 'precinct', how = 'inner')

precincts['precinct'] = precincts['precinct'].astype(str)

# Map Data

In [20]:
# Dictionary to hold graphs keyed on precinct
graphsDict = {}

# For every precinct create a graph
for i in keys:
    # Make a source dataframe using only that precincts complaints
    source = df.loc[df['precinct'] == i]
    # Find all the outcomes for those complaints
    substantiatedCols = source['board_disposition'].unique()
    substantiatedCols = substantiatedCols[(substantiatedCols != 'Exonerated') & (substantiatedCols != 'Unsubstantiated')]
    # Get statistics for each officer's complaints in the precinct
    precinctdf = pd.pivot_table(source,
                                values = ['first_name'],
                                index = ['unique_mos_id'],
                                columns = ['board_disposition'],aggfunc = 'count')['first_name']
    precinctdf = precinctdf.assign(Substantiated = precinctdf[substantiatedCols].sum(1)).drop(substantiatedCols,1)
    precinctdf = precinctdf.fillna(0)
    precinctdf['total'] = precinctdf['Exonerated'] + precinctdf['Unsubstantiated'] + precinctdf['Substantiated']
    precinctdf['percent'] = precinctdf['Substantiated']/precinctdf['total']
    precinctdf = precinctdf.reset_index()

    # Update source dataframe
    source = df.merge(precinctdf,on = 'unique_mos_id', how = 'inner')
    
    # Create selector for graph interaction
    selector = alt.selection_single(empty = 'all', fields = ['unique_mos_id'])

    # Define base graph and add selector
    base = alt.Chart(source).properties(
        width = 300,
        height = 350
    ).add_selection(selector)

    # Make points graph
    points = base.mark_point(filled = True, size = 25).encode(
        x = alt.X('total:Q', axis = alt.Axis(title = 'Total number of complaints at precinct')),
        y = alt.Y('percent:Q', axis = alt.Axis(title = 'Percentage of complaints that were substantiated')),
        tooltip = ['first_name','last_name','Substantiated','Exonerated','Unsubstantiated'],
    )

    # Make histogram and add selector
    hists = base.mark_bar(opacity = 0.5, thickness = 100).encode(
        x = alt.X('year_received:O', axis = alt.Axis(title = 'Year complaints were received')),
        y = alt.Y('count(year_received)', axis = alt.Axis(title = 'Number of complaints')),
        color = 'board_disposition'
    ).transform_filter(
        selector
    )
    
    # Add graphs to dictionary
    graphsDict[i] = points|hists

In [21]:
# Define Folium map and initialize it
choroplethGraph = folium.Map([40.719435, -74.001930], max_zoom = 12)
# Define folium layers for graphs and descriptions
precinctLayer = folium.FeatureGroup(name = 'Graphs')
descriptionLayer = folium.FeatureGroup(name = 'Descriptions', show = False)
# Make the layers transparent so they don't change the choropleth colors
transparent = {'fillColor': '#00000000', 'color': '#00000000'}
# Make a choropleth based on police report data
folium.Choropleth(
    geo_data = precinctMap,
    data = minoritydf,
    columns = ['precinct', 'ratio'],
    key_on = 'feature.properties.precinct',
    fill_color = 'BuPu',
    fill_opacity = 0.7,
    line_opacity = 0.2,
    control = False,
    legend_name = 'Ratio of Police Reports made by Minorities to Overall Police Reports').add_to(choroplethGraph)

# Loop through precincts and attach descriptions and graphs to precinct geography
for i in keys:
    gsP = folium.GeoJson(precinctMap.loc[precinctMap['precinct'] == str(i)])
    folium.Popup(html = str(i), max_width = 900, sticky = True).add_child(
        folium.VegaLite(graphsDict[i], width = 900, height = 400)).add_to(gsP)
    precinctLayer.add_child(gsP)
    
    datarow = precincts.loc[precincts['precinct'] == str(i)].reset_index()
    officerrow = uniqueIDdf.loc[i]
    gs = folium.GeoJson(precinctMap.loc[precinctMap['precinct'] == str(i)], style_function = lambda x: transparent)
    precinctlabel = i
    officerslabel = datarow['Uniformed Officers'][0]
    complaintslabel = datarow['Number of Complaints'][0]
    substantiatedlabel = datarow['Substantiated'][0]
    officersreceivelabel = uniqueIDdf.loc[i]['first_name'].size
    yearmin = datarow['min year'][0]
    yearmax = datarow['max year'][0]
    maxcomplaints = uniqueIDdf.loc[i].iloc[-1]['first_name']
    popuphtml = """
            <html>
                <body>
                    <h1>Precinct %s</h1>
                    <p>Number of Current Uniformed Officers: %s</p>
                    <p>Number of Complaints: %s</p>
                    <p>Number of Substantiated Complaints: %s</p>
                    <p>Number of Officers who have Received Complaints at Precinct %s: %s</p>
                    <p>Years of Complaints: %s - %s</p>
                    <p>Max Complaints for a Single Officer: %s</p>
                </body>
            </html>"""%(precinctlabel, str(officerslabel), str(complaintslabel), str(substantiatedlabel), precinctlabel, 
                        str(officersreceivelabel), str(yearmin), str(yearmax), str(maxcomplaints))
    folium.Popup(popuphtml,max_width = 400, sticky = False).add_to(gs)
    gs.add_to(descriptionLayer)
    
    
# Add groups to folium map    
choroplethGraph.add_child(precinctLayer)
choroplethGraph.add_child(descriptionLayer)
folium.LayerControl().add_to(choroplethGraph)

In [22]:
# Save map to external file
choroplethGraph.save('ChoroplethGraph.html')